In [1]:
import os
from pathlib import Path
import pickle
from fastfusion import Specification
from fastfusion.mapper.FFM.exploration.mapper_multi_einsum import get_sims
from fastfusion.mapper.FFM.exploration.mapper_one_einsum import get_single_einsum_sims
from fastfusion.mapper.FFM.joining.simexplore import decompress_sims, join_sims
from fastfusion.mapper.FFM.exploration.mapping_filter_tags import get_one_split_tag, get_ffmt_tag


spec = Specification.from_yaml(
    "architecture/snowcat.arch.yaml",
    "workloads/matmuls8_mixed.workload.yaml",
)


MAPPINGS_DATA_DIR = Path('results/')


workload = spec.workload
renames = spec.renames

# pr = cProfile.Profile()
# pr.enable()

# sims = get_single_einsum_sims(spec, "Q", rank_variable_bounds)

def one_split_tagger(pmapping):
    return get_one_split_tag(pmapping,
                             workload.intermediate_tensors(),
                             "MainMemory")

NAME_TO_TAGGER = {
    'one_split': one_split_tagger
}


def get_experiment_name(tagger_name):
    return f'matmul8_mixed.{tagger_name}'

def get_sims_with_cache(tagger_name=None, refresh_cache=False):
    experiment_name = get_experiment_name(tagger_name)
    if tagger_name is None:
        tagger = None
    else:
        tagger = NAME_TO_TAGGER[tagger_name]

    result_pickle_name = MAPPINGS_DATA_DIR / f'{experiment_name}.pkl'
    if result_pickle_name.is_file() and not refresh_cache:
        with open(result_pickle_name, 'rb') as f:
            mappings = pickle.load(f)
            print(f'Loaded results from cache {result_pickle_name}')
            return mappings

    spec.estimate_energy_area()
    flattened_architecture = spec.get_flattened_architecture()
    sims, decompress_data = get_sims(spec, flattened_architecture, tagger=tagger)
    mappings = join_sims(sims, spec, flattened_architecture, drop_valid_reservations=False)
    decompress_sims(mappings, decompress_data, workload.einsum_names)

    with open(result_pickle_name, 'wb') as f:
        pickle.dump(mappings, f)
        print(f'Saved results to cache {result_pickle_name}')

    return mappings

mappings_one_split = get_sims_with_cache('one_split', refresh_cache=True)
mappings = get_sims_with_cache()

# pr.disable()
# s = io.StringIO()
# ps = pstats.Stats(pr, stream=s).sort_stats('cumulative')
# ps.print_stats(30)  # Print top 30 time-consuming functions
# print(s.getvalue())

# TODO: Check for ranks not in the mapping and put them at the bottom
# TODO: What if there are no loops? 
# TODO: Set _must_exist for all backing storage nodes
# TODO: Constraint attacher
# TODO: Can't have tile size constraints on backing memory
# TODO: Einsum orders
# TODO: Copy Einsums
# TODO: Test dataflow constraints and order of storage nodes
# I'm doing the tile shape exploration now and I'm trying to understand this note. I think I understand what you're saying.
# Can I ask one thing from the constraint code? If the constraint is an equality, then just set the tile_shape attribute of the node (or factor or whatever is needed) to the value.
# The tile shape exploration assumes a particular mapspace (in most cases, tile shapes are factors of the full rank shape), so an equality may never be satisfied. E.g., if the constraint sets the tile shape equal to a non-factor value because you want a particular imperfect factorization, but that's never in the mapspace, then you'll get nothing.
# It's also a bit more efficient to just set the value and the explorer doesn't have to figure out the equality by trial-and-error. For other more complicated constraints, trial-and-error is better.

INFO        Loading yaml file architecture/snowcat.arch.yaml
INFO        Found top key variables in architecture/snowcat.arch.yaml
INFO        Found top key architecture in architecture/snowcat.arch.yaml
INFO        Loading yaml file workloads/matmuls8_mixed.workload.yaml
INFO        Found top key workload in workloads/matmuls8_mixed.workload.yaml
WARNING     Loading configuration file from /home/gilbertm/work/infrastructure/work-venv/fastfusion/config.yaml
INFO        Calculated "0.5" = 0.5.
Generating storage and loop choices for Einsum Matmul1: 37it [00:00, 1521.61it/s]
Generating storage and loop choices for Einsum Matmul2: 57it [00:00, 1583.31it/s]
Generating storage and loop choices for Einsum Matmul3: 57it [00:00, 1622.71it/s]
Generating storage and loop choices for Einsum Matmul4: 57it [00:00, 1622.65it/s]
Generating storage and loop choices for Einsum Matmul5: 57it [00:00, 1653.94it/s]
Generating storage and loop choices for Einsum Matmul6: 57it [00:00, 1614.54it/s]
Generating

SIM Matmul1 tensors: {'T1'}
SIM Matmul2 tensors: {'T2', 'T1'}
SIM Matmul3 tensors: {'T2', 'T3'}
SIM Matmul4 tensors: {'T4', 'T3'}
SIM Matmul5 tensors: {'T4', 'T5'}
SIM Matmul6 tensors: {'T5', 'T6'}
SIM Matmul7 tensors: {'T7', 'T6'}
SIM Matmul8 tensors: {'T7'}


Inital consolidate Matmul8: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 114/114 [00:00<00:00, 4459.15it/s]


Initial consolidate and group: 1.04 seconds

Einsum Matmul2 (2/8)
Consolidating: 0.00 seconds
Combining: 0.00 seconds
Grouping: 0.00 seconds
Bucket merging: 0.00 seconds
Removed 0/212 (100.00% remaining)
Removing mappings that can't be combined later: 0.00 seconds


Merging mappings Matmul1 <--> Matmul2: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 212/212 [00:00<00:00, 3908.04it/s]


...

Merging mappings Matmul7 <--> Matmul8: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 115/115 [00:00<00:00, 2388.11it/s]


Mapping merging: 0.11 seconds
Scaled runtime by 1.0. Runtime: 0.26
	Combining 210(114) x 210(114) -> 115
	Number of buckets for Einsum Matmul8: 115
	Number of mappings for Einsum Matmul8: 2437
	Mappings per bucket for Einsum Matmul8: 21.191304347826087


Combining SIMs: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 2744.96it/s]


======== Total time ========
Initial consolidate and group: 1.04 seconds
Consolidating: 0.02 seconds
Combining: 0.42 seconds
Grouping: 0.00 seconds
Bucket merging: 0.05 seconds
Removing mappings that can't be combined later: 0.01 seconds
Mapping merging: 1.46 seconds

Total: 3.00 seconds



Saved results to cache results/matmul8_mixed.one_split.pkl
Loaded results from cache results/matmul8_mixed.None.pkl


In [2]:
import copy
import re
from fastfusion.frontend.mapping import Iteration, Mapping, Nested, Split, Storage
from fastfusion.visualization.interactive import plotly_show
from fastfusion.mapper.FFM.visualization import make_mapping

plotly_show(mappings.data, "RESOURCE_GlobalBuffer_LEVEL_0", "metric_Energy", logscales=True, einsum_names=spec.workload.einsum_names)

    'data': [{'line': {'shape': 'hv'},
              'marker': {'symbol': 'circl…

In [3]:
plotly_show(mappings_one_split.data, "RESOURCE_GlobalBuffer_LEVEL_0", "metric_Energy", logscales=True, einsum_names=spec.workload.einsum_names)

    'data': [{'line': {'shape': 'hv'},
              'marker': {'symbol': 'circl…